In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
tournament = pd.read_csv('/kaggle/input/ncaam-march-mania-2021/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
df = pd.read_csv('/kaggle/input/ncaam-march-mania-2021/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
df['Games'] = 1

In [ ]:
end = tournament['Season'].max()
start = df['Season'].min()

tournament = tournament.loc[(end >= tournament['Season']) & (tournament['Season'] >= start)]
df = df.loc[(end >= df['Season']) & (df['Season'] >= start)]

In [ ]:
df

In [ ]:
winners = df.groupby(['Season', 'WTeamID']).agg(
    {'WScore': 'mean', 'WAst': 'mean', 'WTO': 'mean', 'WStl': 'mean', 'WPF': 'mean', 'Games': 'sum'}
)
winners.rename(columns={'Games': 'WGames'}, inplace=True)

In [ ]:
winners

In [ ]:
losers = df.groupby(['Season', 'LTeamID']).agg(
    {'LScore': 'mean', 'LAst': 'mean', 'LTO': 'mean', 'LStl': 'mean', 'LPF': 'mean', 'Games': 'sum'}
)
losers.rename(columns={'Games': 'LGames'}, inplace=True)

In [ ]:
losers

In [ ]:
data = winners.join(losers, on=['Season', 'WTeamID'])

In [ ]:
data

In [ ]:
data['Games'] = data['WGames'] + data['LGames']
data['WPct'] = data['WGames'] / data['Games']
data['LPct'] = data['LGames'] / data['Games']
data['PPG'] = data['WScore'] * data['WPct'] + data['LScore'] * data['LPct']
data['Ast'] = data['WAst'] * data['WPct'] + data['LAst'] * data['LPct']
data['TO'] = data['WTO'] * data['WPct'] + data['LTO'] * data['LPct']
data['Stl'] = data['WStl'] * data['WPct'] + data['LStl'] * data['LPct']
data['PF'] = data['WPF'] * data['WPct'] + data['LPF'] * data['LPct']

In [ ]:
data

In [ ]:
tournament = tournament.set_index('Season')
tournament

In [ ]:
tourney = tournament.join(data[['PPG', 'WPct', 'Ast', 'TO', 'Stl', 'PF']], on=['Season', 'WTeamID'])
tourney.rename(columns={'PPG': 'WPPG', 'WPct': 'WWPct', 'Ast': 'WAst', 'TO': 'WTO', 'Stl': 'WStl', 'PF': 'WPF'}, inplace=True)
tourney = tourney.join(data[['PPG', 'WPct', 'Ast', 'TO', 'Stl', 'PF']], on=['Season', 'LTeamID'])
tourney.rename(columns={'PPG': 'LPPG', 'WPct': 'LWPct', 'Ast': 'LAst', 'TO': 'LTO', 'Stl': 'LStl', 'PF': 'LPF'}, inplace=True)

In [ ]:
tourney

In [ ]:
X, y = [], []
cols = ['PPG', 'WPct', 'Ast', 'Stl', 'TO', 'PF']
w_cols = ['W' + col for col in cols]
l_cols = ['L' + col for col in cols]
for _, row in tourney.iterrows():
    order = np.random.randint(0, 2)
    y.append(order)
    if order:
        X.append(row[w_cols + l_cols].tolist())
    else:
        X.append(row[l_cols + w_cols].tolist())

X = pd.DataFrame(X, columns=['1' + col for col in cols] + ['2' + col for col in cols])
y = pd.Series(y)

In [ ]:
X

In [ ]:
y

In [ ]:
X.isnull().sum(axis=1).tolist()

In [ ]:
not bool(0)

In [ ]:
X.dropna()

In [ ]:
y.dropna()

In [ ]:
bad_apples = X.isnull().sum(axis=1).tolist()
good_apples = [not bool(b) for b in bad_apples]
X = X.loc[good_apples]
y = y.loc[good_apples]

In [ ]:
y

In [ ]:
from statsmodels.api import Logit

In [ ]:
mdl = Logit(y, X).fit()

In [ ]:
mdl.summary()

In [ ]:
different_X = pd.DataFrame(X[['1PPG', '1WPct', '1Ast', '1Stl', '1TO', '1PF']].values - X[['2PPG', '2WPct', '2Ast', '2Stl', '2TO', '2PF']].values, columns=['PPGDelta', 'WPctDelta', 'AstDelta', 'StlDelta', 'TODelta', 'PFDelta'], index=y.index)

In [ ]:
different_X

In [ ]:
mdl = Logit(y, different_X).fit()

In [ ]:
mdl.summary()